### Notebook to generate SWITCH input files for wave/offshore wind collocation runs (set 2 with hybrid projects)

Python Packages

In [169]:
# Third-party packages
import h5py
import h5pyd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from scipy.interpolate import interp1d


from scipy.spatial import cKDTree
from tqdm.notebook import trange, tqdm

Load all files

In [170]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/set1_runs/generation_project_info.csv"
gen_proj_info = pd.read_csv(fpath, index_col=0)
#gen_proj_info['generation_project'] = gen_proj_info['generation_project']+300	
gen_proj_info['gen_dbid'] = gen_proj_info['generation_project']
gen_proj_info

,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long
index,,,,,,,,,,,,,,,,,,
0,1400000000,139.693878,487000,1400000000,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.276111,-124.767222
1,1400000001,1351.628594,487000,1400000001,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.188333,-125.072778
2,1400000002,94.725429,487000,1400000002,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.035278,-124.923056
3,1400000003,108.360230,487000,1400000003,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.922500,-124.702778
4,1400000004,189.654849,487000,1400000004,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.820556,-124.576944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,1400000197,1191.525700,487000,1400000197,Wind,0.05,.,False,False,True,CA_SCE_CEN,30,0,0.06,Offshore_Wind,0,34.070556,-119.185833
198,1400000198,3354.236500,487000,1400000198,Wind,0.05,.,False,False,True,CA_SCE_S,30,0,0.06,Offshore_Wind,0,33.527222,-118.872500
199,1400000199,4194.181300,487000,1400000199,Wind,0.05,.,False,False,True,CA_PGE_S,30,0,0.06,Offshore_Wind,0,35.583733,-121.838928


In [171]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/Finalized files/final_cand_proj_areas_sizing.csv"
cand_proj_areas = pd.read_csv(fpath)
cand_proj_areas.rename(columns={'GENERATION_PROJECT':'generation_project','lon':'long'},inplace=True)
cand_proj_areas

,site_id,generation_project,site_name,state,lat,long,load_zone,area_sq_km,wave_cap_mw,osw_cap_mw,wind_turbine_type
0,0,1400000000,Neah Bay 1,WA,48.276111,-124.767222,WA_W,132.852,139.693878,571.2636,fixed
1,1,1400000001,Neah Bay 2,WA,48.188333,-125.072778,WA_W,1285.429,1351.628594,5527.3447,float
2,2,1400000002,La Push 1,WA,48.035278,-124.923056,WA_W,90.086,94.725429,387.3698,float
3,3,1400000003,La Push 2,WA,47.922500,-124.702778,WA_W,103.053,108.360230,443.1279,fixed
4,4,1400000004,La Push 3,WA,47.820556,-124.576944,WA_W,180.366,189.654849,775.5738,fixed
...,...,...,...,...,...,...,...,...,...,...,...
197,96,1400000197,Port Hueneme 1,CA,34.070556,-119.185833,CA_SCE_CEN,277.099,291.369598,1191.5257,float
198,97,1400000198,Torrance,CA,33.527222,-118.872500,CA_SCE_S,780.055,820.227832,3354.2365,float
199,98,1400000199,Morro Bay_Call,CA,35.583733,-121.838928,CA_PGE_S,975.391,1025.623637,4194.1813,float
200,99,1400000200,San Diego,CA,32.625556,-117.331389,CA_SDGE,1419.395,1492.493843,6103.3985,float


In [172]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/set1_runs/scenario_cost_files/all_scenarios_gen_build_cost_wave_offshore_wind.csv'
cost_scenarios = pd.read_csv(fpath)
cost_scenarios = cost_scenarios.rename(columns={'index':'site_id'})
cost_scenarios

,site_id,generation_project,build_year,gen_fixed_om,gen_overnight_cost,scenario_id
0,0,1400000000,2010,105400.000,3465000.00,1
1,0,1400000000,2011,105400.000,3465000.00,1
2,0,1400000000,2012,105400.000,3465000.00,1
3,0,1400000000,2013,105400.000,3465000.00,1
4,0,1400000000,2014,105400.000,3465000.00,1
...,...,...,...,...,...,...
257545,201,1400000201,2056,39253.484,2728300.17,25
257546,201,1400000201,2057,39253.484,2728300.17,25
257547,201,1400000201,2058,39253.484,2728300.17,25
257548,201,1400000201,2059,39253.484,2728300.17,25


In [173]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/percentages_for_set2_runs.csv'
percents_per_proj = pd.read_csv(fpath)
percents_per_proj

,scenario,site_id,PERIOD,gen_load_zone,wind_turbine_type,wave_cap_mw,GenCapacity_wave,osw_cap_mw,GenCapacity_osw,Total_GW,percent_built_cap_wave,percent_built_cap_osw
0,25,0,2050,WA_W,fixed,139.693878,139.690,571.2636,571.26,0.710950,19.648358,80.351642
1,25,1,2050,WA_W,float,1351.628594,1351.600,5527.3447,4123.00,5.474600,24.688562,75.311438
2,25,2,2050,WA_W,float,94.725429,94.725,387.3698,0.00,0.094725,100.000000,0.000000
3,25,3,2050,WA_W,fixed,108.360230,108.360,443.1279,0.00,0.108360,100.000000,0.000000
4,25,4,2050,WA_W,fixed,189.654849,189.650,775.5738,0.00,0.189650,100.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
89,25,96,2050,CA_SCE_CEN,float,291.369598,291.370,1191.5257,0.00,0.291370,100.000000,0.000000
90,25,97,2050,CA_SCE_S,float,820.227832,820.230,3354.2365,0.00,0.820230,100.000000,0.000000
91,25,98,2050,CA_PGE_S,float,1025.623637,1025.600,4194.1813,0.00,1.025600,100.000000,0.000000
92,25,99,2050,CA_SDGE,float,1492.493843,1492.500,6103.3985,0.00,1.492500,100.000000,0.000000


Sites that had some build out (either wave, osw, or both) from scenario 25 of set 1

In [174]:
sites_to_keep = percents_per_proj['site_id']

generation_project_info file

In [175]:
gen_proj_info_temp = gen_proj_info.merge(cand_proj_areas,how='outer',on=['generation_project','lat','long'])
gen_proj_info_temp = gen_proj_info_temp[['generation_project','gen_capacity_limit_mw','gen_connect_cost_per_mw','gen_dbid','gen_energy_source','gen_forced_outage_rate','gen_full_load_heat_rate','gen_is_baseload','gen_is_cogen','gen_is_variable','gen_load_zone','gen_max_age','gen_min_build_capacity','gen_scheduled_outage_rate','gen_tech','gen_variable_om','lat','long','site_id']]
gen_proj_info_temp

,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long,site_id
0,1400000000,139.693878,487000,1400000000,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.276111,-124.767222,0
1,1400000001,1351.628594,487000,1400000001,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.188333,-125.072778,1
2,1400000002,94.725429,487000,1400000002,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.035278,-124.923056,2
3,1400000003,108.360230,487000,1400000003,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.922500,-124.702778,3
4,1400000004,189.654849,487000,1400000004,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.820556,-124.576944,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,1400000197,1191.525700,487000,1400000197,Wind,0.05,.,False,False,True,CA_SCE_CEN,30,0,0.06,Offshore_Wind,0,34.070556,-119.185833,96
198,1400000198,3354.236500,487000,1400000198,Wind,0.05,.,False,False,True,CA_SCE_S,30,0,0.06,Offshore_Wind,0,33.527222,-118.872500,97
199,1400000199,4194.181300,487000,1400000199,Wind,0.05,.,False,False,True,CA_PGE_S,30,0,0.06,Offshore_Wind,0,35.583733,-121.838928,98
200,1400000200,6103.398500,487000,1400000200,Wind,0.05,.,False,False,True,CA_SDGE,30,0,0.06,Offshore_Wind,0,32.625556,-117.331389,99


In [176]:
gen_proj_info_temp_osw = gen_proj_info_temp[gen_proj_info_temp['gen_tech']=='Offshore_Wind']
gen_proj_info_temp_osw = gen_proj_info_temp_osw.reset_index(drop=True)

gen_proj_info_temp_wave = gen_proj_info_temp[gen_proj_info_temp['gen_tech']=='Wave']
gen_proj_info_temp_wave['generation_project'] = gen_proj_info_temp_wave['generation_project']+300
gen_proj_info_temp_wave['gen_dbid'] = gen_proj_info_temp_wave['generation_project']
gen_proj_info_temp_wave = gen_proj_info_temp_wave[gen_proj_info_temp_wave['site_id'].isin(sites_to_keep)]
gen_proj_info_temp_wave

<ipython-input-176-27a9c04179de>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gen_proj_info_temp_wave['generation_project'] = gen_proj_info_temp_wave['generation_project']+300
<ipython-input-176-27a9c04179de>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gen_proj_info_temp_wave['gen_dbid'] = gen_proj_info_temp_wave['generation_project']


,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long,site_id
0,1400000300,139.693878,487000,1400000300,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.276111,-124.767222,0
1,1400000301,1351.628594,487000,1400000301,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.188333,-125.072778,1
2,1400000302,94.725429,487000,1400000302,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.035278,-124.923056,2
3,1400000303,108.360230,487000,1400000303,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.922500,-124.702778,3
4,1400000304,189.654849,487000,1400000304,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.820556,-124.576944,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1400000396,291.369598,487000,1400000396,Wave,0.07,.,False,False,True,CA_SCE_CEN,20,0,0.01,Wave,0,34.070556,-119.185833,96
97,1400000397,820.227832,487000,1400000397,Wave,0.07,.,False,False,True,CA_SCE_S,20,0,0.01,Wave,0,33.527222,-118.872500,97
98,1400000398,1025.623637,487000,1400000398,Wave,0.07,.,False,False,True,CA_PGE_S,20,0,0.01,Wave,0,35.583733,-121.838928,98
99,1400000399,1492.493843,487000,1400000399,Wave,0.07,.,False,False,True,CA_SDGE,20,0,0.01,Wave,0,32.625556,-117.331389,99


In [177]:
hybrid_cap_limits = gen_proj_info_temp.pivot_table(
    index="site_id", values="gen_capacity_limit_mw", aggfunc=np.sum )
hybrid_cap_limits

,gen_capacity_limit_mw
site_id,
0,710.957478
1,6878.973294
2,482.095229
3,551.488130
4,965.228649
...,...
96,1482.895298
97,4174.464332
98,5219.804937


In [198]:
hybrid_gen_proj_info = gen_proj_info_temp_wave.copy()
for i in sites_to_keep:
    p_wave = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_wave'].reset_index(drop=True)[0]/100
    p_osw = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_osw'].reset_index(drop=True)[0]/100

    if p_osw == 1:
        hybrid_gen_proj_info.loc[i,'gen_capacity_limit_mw'] = gen_proj_info_temp_osw.loc[i,'gen_capacity_limit_mw']
        hybrid_gen_proj_info.loc[i,'gen_energy_source'] = 'Wind'
        hybrid_gen_proj_info.loc[i,'gen_tech'] = 'Offshore_Wind'
        hybrid_gen_proj_info.loc[i,'gen_max_age'] = 30
        hybrid_gen_proj_info.loc[i,'gen_forced_outage_rate'] = 0.05
        hybrid_gen_proj_info.loc[i,'gen_scheduled_outage_rate'] = 0.06
    elif p_wave != 1 and p_osw !=1:
        hybrid_gen_proj_info.loc[i,'gen_capacity_limit_mw'] = hybrid_cap_limits.loc[i,'gen_capacity_limit_mw']
        hybrid_gen_proj_info.loc[i,'gen_energy_source'] = 'Wave_Wind'
        hybrid_gen_proj_info.loc[i,'gen_tech'] = 'Wave_Wind_Hybrid'
        hybrid_gen_proj_info.loc[i,'gen_max_age'] = (20*p_wave) + (30*p_osw)
        hybrid_gen_proj_info.loc[i,'gen_forced_outage_rate'] = (0.07*p_wave) + (0.05*p_osw)
        hybrid_gen_proj_info.loc[i,'gen_scheduled_outage_rate'] = (0.01*p_wave) + (0.06*p_osw)
hybrid_gen_proj_info = hybrid_gen_proj_info.drop(columns={'site_id'})
hybrid_gen_proj_info

,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long
0,1400000300,710.957478,487000,1400000300,Wave_Wind,0.053930,.,False,False,True,WA_W,28.035164,0,0.050176,Wave_Wind_Hybrid,0,48.276111,-124.767222
1,1400000301,6878.973294,487000,1400000301,Wave_Wind,0.054938,.,False,False,True,WA_W,27.531144,0,0.047656,Wave_Wind_Hybrid,0,48.188333,-125.072778
2,1400000302,94.725429,487000,1400000302,Wave,0.070000,.,False,False,True,WA_W,20.000000,0,0.010000,Wave,0,48.035278,-124.923056
3,1400000303,108.360230,487000,1400000303,Wave,0.070000,.,False,False,True,WA_W,20.000000,0,0.010000,Wave,0,47.922500,-124.702778
4,1400000304,189.654849,487000,1400000304,Wave,0.070000,.,False,False,True,WA_W,20.000000,0,0.010000,Wave,0,47.820556,-124.576944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1400000396,291.369598,487000,1400000396,Wave,0.070000,.,False,False,True,CA_SCE_CEN,20.000000,0,0.010000,Wave,0,34.070556,-119.185833
97,1400000397,820.227832,487000,1400000397,Wave,0.070000,.,False,False,True,CA_SCE_S,20.000000,0,0.010000,Wave,0,33.527222,-118.872500
98,1400000398,1025.623637,487000,1400000398,Wave,0.070000,.,False,False,True,CA_PGE_S,20.000000,0,0.010000,Wave,0,35.583733,-121.838928
99,1400000399,1492.493843,487000,1400000399,Wave,0.070000,.,False,False,True,CA_SDGE,20.000000,0,0.010000,Wave,0,32.625556,-117.331389


In [199]:
hybrid_gen_proj_info.to_csv('generation_project_info.csv',index=False)

Capacity factor time series file

In [180]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/finalized files/wave_data.csv"
wave_ts = pd.read_csv(fpath)
wave_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000000,0.594093,0,2006,1,1,0
1,1400000000,0.610109,0,2006,1,1,1
2,1400000000,0.626125,0,2006,1,1,2
3,1400000000,0.642141,0,2006,1,1,3
4,1400000000,0.655864,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000100,0.520220,100,2006,12,31,19
884756,1400000100,0.507486,100,2006,12,31,20
884757,1400000100,0.494751,100,2006,12,31,21
884758,1400000100,0.494751,100,2006,12,31,22


In [181]:
wave_ts_reduced = wave_ts[wave_ts['site'].isin(sites_to_keep)]
wave_ts_reduced['generation_project'] = wave_ts_reduced['generation_project']+300
wave_ts_reduced

<ipython-input-181-fe57e9751190>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_ts_reduced['generation_project'] = wave_ts_reduced['generation_project']+300


,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000300,0.594093,0,2006,1,1,0
1,1400000300,0.610109,0,2006,1,1,1
2,1400000300,0.626125,0,2006,1,1,2
3,1400000300,0.642141,0,2006,1,1,3
4,1400000300,0.655864,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000400,0.520220,100,2006,12,31,19
884756,1400000400,0.507486,100,2006,12,31,20
884757,1400000400,0.494751,100,2006,12,31,21
884758,1400000400,0.494751,100,2006,12,31,22


In [182]:
wave_cf_d = {i: wave_ts_reduced[wave_ts_reduced['site']==i] for i in sites_to_keep}

In [183]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/finalized files/offshore_data.csv"
osw_ts = pd.read_csv(fpath)
osw_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000101,0.792742,0,2006,1,1,0
1,1400000101,0.924306,0,2006,1,1,1
2,1400000101,0.536871,0,2006,1,1,2
3,1400000101,0.648541,0,2006,1,1,3
4,1400000101,0.562322,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000201,0.497244,100,2006,12,31,19
884756,1400000201,0.502999,100,2006,12,31,20
884757,1400000201,0.585540,100,2006,12,31,21
884758,1400000201,0.811253,100,2006,12,31,22


In [184]:
osw_ts_reduced = osw_ts[osw_ts['site'].isin(sites_to_keep)]
osw_ts_reduced['generation_project'] = osw_ts_reduced['generation_project']+199
osw_ts_reduced

<ipython-input-184-118bf9e42976>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osw_ts_reduced['generation_project'] = osw_ts_reduced['generation_project']+199


,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000300,0.792742,0,2006,1,1,0
1,1400000300,0.924306,0,2006,1,1,1
2,1400000300,0.536871,0,2006,1,1,2
3,1400000300,0.648541,0,2006,1,1,3
4,1400000300,0.562322,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000400,0.497244,100,2006,12,31,19
884756,1400000400,0.502999,100,2006,12,31,20
884757,1400000400,0.585540,100,2006,12,31,21
884758,1400000400,0.811253,100,2006,12,31,22


In [185]:
osw_cf_d = {i: osw_ts_reduced[osw_ts_reduced['site']==i] for i in sites_to_keep}

In [186]:
hybrid_cf_d = wave_cf_d.copy()
for i in sites_to_keep:
    p_wave = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_wave'].reset_index(drop=True)[0]/100
    p_osw = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_osw'].reset_index(drop=True)[0]/100

    if p_osw == 1:
        hybrid_cf_d[i] = osw_cf_d[i]
    elif p_wave != 1 and p_osw !=1:
        hybrid_cf_d[i]['gen_max_capacity_factor'] = (wave_cf_d[i]['gen_max_capacity_factor']*p_wave) + (osw_cf_d[i]['gen_max_capacity_factor']*p_osw)
wave_cf_d = {i: wave_ts_reduced[wave_ts_reduced['site']==i] for i in sites_to_keep}

<ipython-input-186-c70b59e91b75>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hybrid_cf_d[i]['gen_max_capacity_factor'] = (wave_cf_d[i]['gen_max_capacity_factor']*p_wave) + (osw_cf_d[i]['gen_max_capacity_factor']*p_osw)


In [187]:
hybrid_cfs = pd.concat(hybrid_cf_d.values(),ignore_index=True)
hybrid_cfs['month'] = hybrid_cfs['month'].apply(lambda x: '{0:0>2}'.format(x))
hybrid_cfs['day'] = hybrid_cfs['day'].apply(lambda x: '{0:0>2}'.format(x))
hybrid_cfs['hour'] = hybrid_cfs['hour'].apply(lambda x: '{0:0>2}'.format(x))
hybrid_cfs

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000300,0.753711,0,2006,01,01,00
1,1400000300,0.862572,0,2006,01,01,01
2,1400000300,0.554408,0,2006,01,01,02
3,1400000300,0.647284,0,2006,01,01,03
4,1400000300,0.580701,0,2006,01,01,04
...,...,...,...,...,...,...,...
823435,1400000400,0.520220,100,2006,12,31,19
823436,1400000400,0.507486,100,2006,12,31,20
823437,1400000400,0.494751,100,2006,12,31,21
823438,1400000400,0.494751,100,2006,12,31,22


In [188]:
hybrid_cfs.to_csv('nati_hybrid_wave_osw_cf_v2.csv',index=False)

Cost scenario files

In [189]:
cost_scenarios_wave = cost_scenarios[cost_scenarios['generation_project']<1400000101]
cost_scenarios_wave_reduced = cost_scenarios_wave[cost_scenarios_wave['site_id'].isin(sites_to_keep)]
cost_scenarios_wave_reduced['generation_project'] = cost_scenarios_wave_reduced['generation_project']+300
cost_scenarios_wave_reduced

<ipython-input-189-a81fb758a15b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_scenarios_wave_reduced['generation_project'] = cost_scenarios_wave_reduced['generation_project']+300


,site_id,generation_project,build_year,gen_fixed_om,gen_overnight_cost,scenario_id
0,0,1400000300,2010,105400.0,3465000.0,1
1,0,1400000300,2011,105400.0,3465000.0,1
2,0,1400000300,2012,105400.0,3465000.0,1
3,0,1400000300,2013,105400.0,3465000.0,1
4,0,1400000300,2014,105400.0,3465000.0,1
...,...,...,...,...,...,...
252394,100,1400000400,2056,8000.0,400000.0,25
252395,100,1400000400,2057,8000.0,400000.0,25
252396,100,1400000400,2058,8000.0,400000.0,25
252397,100,1400000400,2059,8000.0,400000.0,25


In [190]:
cost_scenarios_osw = cost_scenarios[cost_scenarios['generation_project']>1400000100]
cost_scenarios_osw['site_id'] = cost_scenarios_osw['site_id']-101
cost_scenarios_osw_reduced = cost_scenarios_osw[cost_scenarios_osw['site_id'].isin(sites_to_keep)]
cost_scenarios_osw_reduced['generation_project'] = cost_scenarios_osw_reduced['generation_project']+199
cost_scenarios_osw_reduced.reset_index(drop=True,inplace=True)
cost_scenarios_osw_reduced

<ipython-input-190-51aef441fcd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_scenarios_osw['site_id'] = cost_scenarios_osw['site_id']-101
<ipython-input-190-51aef441fcd7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_scenarios_osw_reduced['generation_project'] = cost_scenarios_osw_reduced['generation_project']+199


,site_id,generation_project,build_year,gen_fixed_om,gen_overnight_cost,scenario_id
0,0,1400000300,2010,125544.8536,4077691.887,1
1,0,1400000300,2011,125544.8536,4077691.887,1
2,0,1400000300,2012,125544.8536,4077691.887,1
3,0,1400000300,2013,125544.8536,4077691.887,1
4,0,1400000300,2014,125544.8536,4077691.887,1
...,...,...,...,...,...,...
119845,100,1400000400,2056,39253.4840,2728300.170,25
119846,100,1400000400,2057,39253.4840,2728300.170,25
119847,100,1400000400,2058,39253.4840,2728300.170,25
119848,100,1400000400,2059,39253.4840,2728300.170,25


In [191]:
cost_scenarios_wave_d = {i: cost_scenarios_wave_reduced[cost_scenarios_wave_reduced['site_id']==i] for i in sites_to_keep}
cost_scenarios_osw_d = {i: cost_scenarios_osw_reduced[cost_scenarios_osw_reduced['site_id']==i] for i in sites_to_keep}

In [192]:
hybrid_cost_scenarios_d = cost_scenarios_wave_d.copy()
for i in sites_to_keep:
    p_wave = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_wave'].reset_index(drop=True)[0]/100
    p_osw = percents_per_proj[percents_per_proj['site_id']==i]['percent_built_cap_osw'].reset_index(drop=True)[0]/100

    if p_osw == 1:
        hybrid_cost_scenarios_d[i] = cost_scenarios_osw_d[i].copy()
    elif p_wave != 1 and p_osw !=1:
        hybrid_cost_scenarios_d[i] = hybrid_cost_scenarios_d[i].merge(cost_scenarios_wave_d[i],on=['generation_project','site_id','scenario_id','build_year'],suffixes=('_hybrid','_wave'))
        hybrid_cost_scenarios_d[i] = hybrid_cost_scenarios_d[i].merge(cost_scenarios_osw_d[i],on=['generation_project','site_id','scenario_id','build_year'],suffixes=('_hybrid','_osw'))
        hybrid_cost_scenarios_d[i]['om_weighted'] = hybrid_cost_scenarios_d[i].apply(lambda x: x['gen_fixed_om_wave']*p_wave+x['gen_fixed_om']*p_osw,axis=1)
        hybrid_cost_scenarios_d[i]['overnight_weighted'] = hybrid_cost_scenarios_d[i].apply(lambda x: x['gen_overnight_cost_wave']*p_wave+x['gen_overnight_cost']*p_osw,axis=1)
        hybrid_cost_scenarios_d[i] = hybrid_cost_scenarios_d[i][['site_id','generation_project','build_year','om_weighted','overnight_weighted','scenario_id']]
        hybrid_cost_scenarios_d[i] = hybrid_cost_scenarios_d[i].rename(columns={'om_weighted':'gen_fixed_om','overnight_weighted':'gen_overnight_cost'})

cost_scenarios_wave_d = {i: cost_scenarios_wave_reduced[cost_scenarios_wave_reduced['site_id']==i] for i in sites_to_keep}

In [193]:
hybrid_cost_scenarios_df = pd.concat(hybrid_cost_scenarios_d.values(),ignore_index=True)
hybrid_cost_scenarios_df = hybrid_cost_scenarios_df.sort_values(by=['scenario_id','site_id'])
hybrid_cost_scenarios_df

,site_id,generation_project,build_year,gen_fixed_om,gen_overnight_cost,scenario_id
0,0,1400000300,2010,121586.72068,3.957308e+06,1
1,0,1400000300,2011,121586.72068,3.957308e+06,1
2,0,1400000300,2012,121586.72068,3.957308e+06,1
3,0,1400000300,2013,121586.72068,3.957308e+06,1
4,0,1400000300,2014,121586.72068,3.957308e+06,1
...,...,...,...,...,...,...
119845,100,1400000400,2056,8000.00000,4.000000e+05,25
119846,100,1400000400,2057,8000.00000,4.000000e+05,25
119847,100,1400000400,2058,8000.00000,4.000000e+05,25
119848,100,1400000400,2059,8000.00000,4.000000e+05,25


In [194]:
hybrid_cost_scenarios_df.to_csv('all_scenarios_gen_build_cost_wave_osw_hybrid_set2.csv',index=False)